In [1]:
import os
import multiprocessing as mp
import pandas as pd
import scipy.io as sio

from models import Compute_RateMatrix_MISAEx_parallel
from models import Compute_RateMatrix_MISAEx
from RateMatrix_Calcs import calc_probvec_prob2d

In [2]:
def _workers_count():
    cpu_count = 0
    try:
        cpu_count = len(os.sched_getaffinity(0))
    except AttributeError:
        cpu_count = os.cpu_count()
    return cpu_count

### Parallel

In [3]:
trialpath = '/Users/camerongallivan/Research/Hybrid_PBN_Project/Compute_Rate_Matrix_Python/outputs_tmp/Trial_0001-py'
outputPath = os.path.abspath(trialpath)
paramPath = os.path.join(outputPath, 'paramValues.csv')

In [4]:
def parallel_wrapper(row):
    # Building Paths
    rateMatrixPath = os.path.join(row[-1], 'RateMatrix')
    eigenValuesPath =  os.path.join(row[-1], 'EigenValues')
    probVecPath =  os.path.join(row[-1], 'ProbVec')
    prob2DPath = os.path.join(row[-1], 'Prob2D')
    timeScalesPath = os.path.join(row[-1], 'TimeScales')
    saveFileName = 'set_{:05}.mat'.format(row[0])
    
    # Calculating
    RateMatrix, Dimensions, StatesDict = Compute_RateMatrix_MISAEx_parallel.main(row)
    eigenValues, probVec, prob2D, timeScales = calc_probvec_prob2d(RateMatrix, Dimensions)
    # Saving
    sio.savemat(os.path.join(rateMatrixPath, saveFileName), {'RateMatrix': RateMatrix, 'Dimensions': Dimensions})
    sio.savemat(os.path.join(eigenValuesPath, saveFileName), {'EigenValues': eigenValues})
    sio.savemat(os.path.join(probVecPath, saveFileName), {'ProbVec': probVec})
    sio.savemat(os.path.join(prob2DPath, saveFileName), {'Prob2d': prob2D})
    sio.savemat(os.path.join(timeScalesPath, saveFileName), {'TimeScales': timeScales})

In [5]:
# Workers
num_workers = _workers_count()

In [6]:
%%time
#def SimulationWrapper1(paramPath, outputPath):
parametersDF = pd.read_csv(paramPath, index_col=0)
parametersDF['OutputPath'] = outputPath

tups = parametersDF.itertuples(name=None)

# Initiating pool
pool1 = mp.Pool(processes=num_workers)
results1 = pool1.map(parallel_wrapper, tups)

# Not including for now
# np.save(os.path.join(outputPath, 'StatesDict.npy'), StatesDict)

CPU times: user 65.4 ms, sys: 37.9 ms, total: 103 ms
Wall time: 36 s


In [11]:
%%time
#def SimulationWrapper1(paramPath, outputPath):
parametersDF = pd.read_csv(paramPath, index_col=0)
parametersDF['OutputPath'] = outputPath

tups = parametersDF.itertuples(name=None)

# Workers
num_workers = _workers_count()

# Initiating pool
pool2 = mp.Pool(processes=num_workers)
results = pool2.map_async(parallel_wrapper, tups).get()

CPU times: user 139 ms, sys: 73.6 ms, total: 213 ms
Wall time: 35.7 s


In [12]:
%%time
parametersDF = pd.read_csv(paramPath, index_col=0)

rateMatrixPath = os.path.join(outputPath, 'RateMatrix')
eigenValuesPath =  os.path.join(outputPath, 'EigenValues')
probVecPath =  os.path.join(outputPath, 'ProbVec')
prob2DPath = os.path.join(outputPath, 'Prob2D')
timeScalesPath = os.path.join(outputPath, 'TimeScales')

for row in parametersDF.itertuples():
    #print(row)
    saveFileName = 'set_{:05}.mat'.format(row.Index)

    # Calculating and saving rate matrix
    RateMatrix, Dimensions, StatesDict = Compute_RateMatrix_MISAEx.main(row)
    eigenValues, probVec, prob2D, timeScales = calc_probvec_prob2d(RateMatrix, Dimensions)
    
    sio.savemat(os.path.join(rateMatrixPath, saveFileName), {'RateMatrix': RateMatrix, 'Dimensions': Dimensions})    
    sio.savemat(os.path.join(eigenValuesPath, saveFileName), {'EigenValues': eigenValues})
    sio.savemat(os.path.join(probVecPath, saveFileName), {'ProbVec': probVec})
    sio.savemat(os.path.join(prob2DPath, saveFileName), {'Prob2d': prob2D})
    sio.savemat(os.path.join(timeScalesPath, saveFileName), {'TimeScales': timeScales})

CPU times: user 55 s, sys: 926 ms, total: 55.9 s
Wall time: 36.3 s
